### 1. Lógica de NN para el 2 armed bandit

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# ======= 1. ENTORNO: Bandit de 2 brazos =======
class TwoArmedBandit:
    def __init__(self):
        # Cada episodio, la probabilidad para el brazo 0 se elige aleatoriamente en [0.1, 0.9].
        # El brazo 1 tendrá probabilidad = 1 - p
        p = np.random.uniform(0.1, 0.9)
        self.probs = [p, 1 - p]
    
    def pull(self, action):
        # Regresa 1 con la probabilidad correspondiente o 0
        return 1 if np.random.rand() < self.probs[action] else 0

# ======= 2. MODELO: Agente PPO con LSTM =======
class PPOAgent(nn.Module):
    def __init__(self, input_size=4, hidden_size=32, num_actions=2):
        """
        input_size: dimensión del vector de entrada. Aquí se usa one-hot para la acción (2),
                    la recompensa previa (1) y el timestep (1) => 2+1+1=4.
        """
        super(PPOAgent, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTMCell(input_size, hidden_size)
        self.policy_head = nn.Linear(hidden_size, num_actions)  # salida de logits para 2 acciones
        self.value_head = nn.Linear(hidden_size, 1)             # salida de valor

    def reset_state(self):
        self.hx = torch.zeros(1, self.hidden_size)
        self.cx = torch.zeros(1, self.hidden_size)

    def forward(self, x):
        # x es de tamaño (1, input_size)
        self.hx, self.cx = self.lstm(x, (self.hx, self.cx))
        logits = self.policy_head(self.hx)
        value = self.value_head(self.hx)
        return logits, value

### 2. Entrenamiento con PPO

In [ ]:
# ======= 3. Crear la entrada del agente =======
# Formato de LSTM 
def get_input(last_action, last_reward, timestep, num_actions=2):
    # Se construye un vector que contiene:
    # - la acción previa en formato one-hot (dim=2)
    # - la recompensa previa (dim=1)
    # - el timestep normalizado (dim=1) (dividido por 10, para mantener magnitudes similares)
    action_one_hot = F.one_hot(torch.tensor([last_action]), num_classes=num_actions).float()
    reward_tensor = torch.tensor([[last_reward]], dtype=torch.float32)
    timestep_tensor = torch.tensor([[timestep / 10.0]], dtype=torch.float32)
    x = torch.cat([action_one_hot, reward_tensor, timestep_tensor], dim=1)
    return x

# ======= 4. HIPERPARÁMETROS DE PPO =======
gamma = 0.99           # factor de descuento
clip_epsilon = 0.2     # parámetro de recorte PPO
ppo_epochs = 4         # número de épocas por actualización
lr = 0.009             # tasa de aprendizaje (número mágico que aprendí en la concentración)

agent = PPOAgent()
optimizer = optim.Adam(agent.parameters(), lr=lr)

num_episodes = 1000    # cantidad total de episodios
episode_length = 5     # pasos por episodio

# ======= 5. CICLO DE ENTRENAMIENTO CON PPO =======
for episode in range(num_episodes):
    env = TwoArmedBandit()
    agent.reset_state()
    
    # Listas para almacenar la trayectoria
    states = []
    actions = []
    rewards = []
    log_probs = []
    values = []
    
    # Inicializamos con una acción por defecto (0) y recompensa 0 para el primer paso
    last_action = 0
    last_reward = 0.0
    
    # Recorrido del episodio
    for t in range(episode_length):
        x = get_input(last_action, last_reward, t)
        logits, value = agent(x)
        probs = F.softmax(logits, dim=1)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        
        # Almacenamos los datos para esta transición
        states.append(x)
        actions.append(action)
        log_probs.append(log_prob)
        values.append(value)
        
        # Tomamos la acción en el entorno y obtenemos la recompensa
        reward = env.pull(action.item())
        rewards.append(reward)
        
        last_action = action.item()
        last_reward = reward
    
    # ======= 5.1. Calcular los RETURNS y las VENTAJAS =======
    returns = []
    G = 0
    for r in reversed(rewards):
        G = r + gamma * G
        returns.insert(0, G)
    returns = torch.tensor(returns, dtype=torch.float32).unsqueeze(1)  # dimensión (episode_length, 1)
    values = torch.cat(values)  # dimensión (episode_length, 1)
    advantages = returns - values.detach()
    
    old_log_probs = torch.cat(log_probs).detach()
    
    # ======= 5.2. Actualización PPO sobre la trayectoria recogida =======
    for _ in range(ppo_epochs):
        new_log_probs = []
        new_values = []
        agent.reset_state()  # Reiniciamos el estado para evaluar la trayectoria almacenada
        
        # Se reevalúa cada estado almacenado
        for i, x in enumerate(states):
            logits, value = agent(x)
            probs = F.softmax(logits, dim=1)
            dist = torch.distributions.Categorical(probs)
            new_log_probs.append(dist.log_prob(actions[i]))
            new_values.append(value)
        new_log_probs = torch.cat(new_log_probs)
        new_values = torch.cat(new_values)
        
        ratio = torch.exp(new_log_probs - old_log_probs)
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1 - clip_epsilon, 1 + clip_epsilon) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()
        value_loss = F.mse_loss(new_values, returns)
        loss = policy_loss + 0.5 * value_loss  # combinación de pérdidas
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (episode+1) % 100 == 0:
        total_reward = sum(rewards)
        print(f"Episode {episode+1}, Total Reward: {total_reward}, Loss: {loss.item():.4f}")

print("Entrenamiento completado.")